In [1]:
import pandas as pd

In [53]:
# Read geospatial data

sez = pd.read_csv("joined_sez_map.csv")

In [52]:
# Read and manipulation of the population data

pop = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/population/p_landscan_pop.csv")
pop.rename(columns = {"asdf_id" : "id"}, inplace=True)
column_filter = pop[pop.columns[pop.columns.str.contains("land")]]

for col in column_filter:
    new_name = "pop" + col.split(".", 1)[1]
    pop.rename(columns = {col : new_name}, inplace = True)

pop_sum = pop[pop.columns[pop.columns.str.contains("sum|id")]]
pop_sum.columns = pop_sum.columns.str.replace(".sum$", "", regex=True)

#pop_sum.dtypes

In [55]:
# Read and manipulation of hte precipitation data

#prep = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm2/precipitation/precipitation_00_22.csv")
#prep_filter = prep[prep.columns[prep.columns.str.contains("pre_yearly")]]

#for col in prep_filter: 
#    new_name = "prep" + col.split(".", 1)[1]
#    prep.rename(columns = {col : new_name}, inplace = True)

#prep.columns = prep.columns.str.replace(".mean$", "", regex=True)
#prep.rename(columns = {"asdf_id" : "id"}, inplace=True)
#prep = prep[prep.columns[prep.columns.str.contains("prep|id")]]

In [58]:
# Read and manipulation of the temperature data

temp = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/temperature/p_landsurface_temp.csv")
temp.rename(columns = {"asdf_id" : "id"}, inplace=True)
temp_filter = temp[temp.columns[temp.columns.str.contains("mod11c3_061")]]
temp_filter = temp_filter[temp_filter.columns[temp_filter.columns.str.contains(".mean.1")]]

for col in temp_filter: 
    new_name = "temp" + col.split(".", 1)[1]
    temp.rename(columns = {col : new_name}, inplace=True)

temp.columns = temp.columns.str.replace(".mean.1", "", regex=True)
temp = temp[temp.columns[temp.columns.str.contains("temp|id")]]
# temp.columns

In [60]:
# Read and manipulation of the Agriculture land cover data

agri = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/landcover/agriculture_land_wide_adm1.csv")
agri.rename(columns = {"provinceCode" : "id"}, inplace=True)
agri = agri[agri.columns[agri.columns.str.contains("agr|id")]]
#agri.columns

In [64]:
# Read and manupulation of the Urban land cover data

urba = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/landcover/urban_land_wide_adm1.csv")
urba.rename(columns = {"provinceCode" : "id"}, inplace=True)
urba = urba[urba.columns[urba.columns.str.contains("urb|id")]]
#urba.columns

In [67]:
# Read and manipulation of the NPP NTL data 

ntl = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/ntl/wide_ntl_adm1_2000_2021.csv")
ntl = ntl[ntl.columns[ntl.columns.str.contains("ntl|id")]]
#ntl.columns

In [70]:
# Read and manipulation of the Electricity consumption revised GDP 

egdp = pd.read_csv("/Users/hendrixperalta/Desktop/Research Data Manipulation/research_data/adm1/ntl/egdp_sum_wide_00-16.csv")
egdp = egdp[egdp.columns[egdp.columns.str.contains("egdp|id")]]
#egdp.columns

In [75]:
# Unifies the different datasets 

datasets = [temp, agri, urba, ntl, egdp]
municipality_data = pd.merge(sez, pop_sum[pop_sum.columns[pop_sum.columns.str.contains("pop|id")]], on="id", how="outer")

for dataset in datasets: 
    municipality_data = pd.merge(municipality_data, dataset, on="id", how= "outer")



# Filters out the years from 2017 to 2022

filter_year = ["2017", "2018", "2018", "2019", "2020", "2021", "2022", "2023","1999"]

pattern = "|".join(filter_year)
municipality_data = municipality_data.drop(columns = municipality_data.columns[municipality_data.columns.str.contains(pattern)])

In [77]:
var_names = ["inv", "ob_f_","tec_f_","adm_f_","ob_m_","tec_m_","adm_m_",'ent', 'ele', "com", 
"inf", "wat", "tss", "buiilt", "ocu", "sal_op", "sal_tec", "pop", "prep", "temp", "agr_", "urb_", "ntl_"]

pattern2 = "|".join(var_names)
long_df = pd.wide_to_long(municipality_data, stubnames=var_names, i='id', j='year', sep='')
long_df = long_df[long_df.columns[long_df.columns.str.contains("shape|id|year")|long_df.columns.str.contains(pattern2)]]
long_df =  long_df.drop(columns = "sez_id")


In [76]:
# Exports the data in a long format 

long_df.to_csv("long_province_sez.csv")